Future Reference?? https://github.com/pcchencode/Final-Project-of-ML-Techniques

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
import pandas as pd
import numpy as np
import csv
from collections import Counter

# Import File From Google Drive

In [122]:
train = pd.read_csv('/content/drive/MyDrive/MachineLearning/Data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MachineLearning/Data/test.csv')

In [ ]:
train['Churn Category'].isna().sum(), test['Churn Category'].isna().sum()

(1408, 1409)

In [123]:
ava_train = train[ train['Churn Category'].notna() ]

In [ ]:
ava_train.head(5)

,Satisfaction Score,Contract,Tenure in Months,Number of Referrals,Total Long Distance Charges,Number of Dependents,Monthly Charge,Age,Churn Category
0,1.0,12.0,35.0,0.0,892.85,0.0,85.15,37.0,1.0
1,1.0,1.0,1.0,0.0,0.00,0.0,35.90,54.0,1.0
2,NaN,12.0,35.0,4.0,1435.70,NaN,20.05,NaN,0.0
3,3.0,1.0,7.0,0.0,NaN,0.0,74.65,23.0,1.0
4,NaN,24.0,68.0,8.0,1164.16,0.0,25.80,41.0,0.0


In [ ]:
ava_train.describe()

In [ ]:
Counter(ava_train['Churn Category'])

Counter({0.0: 3118, 1.0: 490, 2.0: 177, 3.0: 201, 4.0: 124, 5.0: 116})

In [ ]:
# ava_train = ava_train.sample(frac=1).reset_index(drop=True)
ava_train = ava_train.sample(frac=1)

In [ ]:
types = []

for i in range(6):
    types.append( ava_train[ava_train['Churn Category']==i].sample(n=116) )

# type0 = ava_train[ava_train['Churn Category']==0].sample(n=116)
# type1 = ava_train[ava_train['Churn Category']==1].sample(n=116)
# type2 = ava_train[ava_train['Churn Category']==2].sample(n=116)
# type3 = ava_train[ava_train['Churn Category']==3].sample(n=116)
# type4 = ava_train[ava_train['Churn Category']==4].sample(n=116)
# type5 = ava_train[ava_train['Churn Category']==5].sample(n=116)

In [ ]:
for i, type in enumerate(types):
    if i==0:
        ava_train = type
    else:
        ava_train = pd.concat([ava_train, type])
        # ava_train.append(type)

# SVM (CSIE)

In [ ]:
!pip install -U libsvm-official

In [125]:
import requests
import numpy as np
import random
import time
import scipy
from scipy import linalg
from sklearn.impute import SimpleImputer

# from liblinear.liblinearutil import *
from libsvm.svmutil import *

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/MachineLearning/Selected/only_conti.csv').drop(columns=['Unnamed: 0'])
ava_train = train[ train['Churn Category'].notna() ]
ava_train = ava_train.sample(frac=1)

In [126]:
train = pd.read_csv('/content/drive/MyDrive/MachineLearning/Selected/conti_bin.csv').drop(columns=['Unnamed: 0'])
ava_train = train[ train['Churn Category'].notna() ]
ava_train = ava_train.sample(frac=1)

In [75]:
train = pd.read_csv('/content/drive/MyDrive/MachineLearning/Selected/Borderline_smote.csv').drop(columns=['Unnamed: 0'])
ava_train = train[ train['Churn Category'].notna() ]
ava_train = ava_train.sample(frac=1)

In [127]:
ava_train.head(5)

,Satisfaction Score,Contract,Tenure in Months,Number of Referrals,Total Long Distance Charges,Number of Dependents,Monthly Charge,Age,Bin Internet Service,Internet Type_None,HasDependents,Internet Type_Fiber Optic,Offer_Offer E,Bin Unlimited Data,City_San Diego,Payment Method_Credit Card,Bin Online Security,Bin Paperless Billing,Churn Category
2028,3.0,12.0,22.0,0.0,0.00,0.0,43.75,51.0,1.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,2.0
3054,4.0,1.0,49.0,6.0,1392.58,NaN,52.15,19.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0
4709,4.0,1.0,2.0,10.0,68.76,0.0,20.00,45.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
4976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
4479,4.0,1.0,26.0,0.0,0.00,2.0,45.80,51.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [140]:
train = pd.read_csv('/content/drive/MyDrive/MachineLearning/Data/train.csv')[['Customer ID','Satisfaction Score','Contract','Number of Referrals',
                                        'Latitude','City_San Diego','Tenure in Months',
                                        'Bin Online Security', 'Internet Type_None', 'Age', 'Churn Category']]

ava_train = train[ train['Churn Category'].notna() ]
ava_train = ava_train.sample(frac=1)

test = pd.read_csv('/content/drive/MyDrive/MachineLearning/Data/test.csv')[['Customer ID','Satisfaction Score','Contract','Number of Referrals',
                                        'Latitude','City_San Diego','Tenure in Months',
                                        'Bin Online Security', 'Internet Type_None', 'Age', 'Churn Category']]

In [129]:
ava_train

,Customer ID,Satisfaction Score,Contract,Number of Referrals,Latitude,City_San Diego,Tenure in Months,Bin Online Security,Internet Type_None,Age,Churn Category
957,3612-YVGSJ,2.0,1.0,0.0,NaN,NaN,1.0,0.0,0.0,62.0,4.0
5283,2569-WGERO,4.0,1.0,9.0,39.134075,0.0,51.0,0.0,NaN,40.0,0.0
4066,4878-BUNFV,3.0,1.0,0.0,35.735694,0.0,3.0,0.0,0.0,NaN,0.0
5304,1449-XQEMT,3.0,1.0,0.0,34.727529,0.0,2.0,0.0,NaN,22.0,0.0
3576,0277-BKSQP,NaN,12.0,7.0,36.793601,0.0,12.0,1.0,NaN,54.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5611,4154-AQUGT,NaN,NaN,NaN,40.704293,0.0,NaN,NaN,NaN,26.0,4.0
1703,0174-QRVVY,2.0,24.0,0.0,NaN,NaN,67.0,1.0,0.0,25.0,5.0
1677,6585-WCEWR,NaN,24.0,1.0,37.306612,0.0,55.0,1.0,0.0,28.0,1.0
5447,2101-RANCD,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,0.0


In [130]:
ava_train.shape

(4226, 11)

In [131]:
# for "10 features"
imp = SimpleImputer( missing_values = np.nan, strategy='constant' )
imp = imp.fit(ava_train.loc[:,'Satisfaction Score':'Age'])
X_imp_conti = imp.transform(ava_train.loc[:,'Satisfaction Score':'Age'])

In [132]:
ava_train.loc[:,'Satisfaction Score':'Age'].columns

Index(['Satisfaction Score', 'Contract', 'Number of Referrals', 'Latitude',
       'City_San Diego', 'Tenure in Months', 'Bin Online Security',
       'Internet Type_None', 'Age'],
      dtype='object')

In [87]:
# only continuous
imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(ava_train.loc[:,'Satisfaction Score':'Age'])
X_imp_conti = imp.transform(ava_train.loc[:,'Satisfaction Score':'Age'])

In [88]:
# only binary
imp = SimpleImputer( missing_values = np.nan, strategy='most_frequent' )
imp = imp.fit(ava_train.loc[:,'Bin Internet Service':'Bin Paperless Billing'])
X_imp_bin = imp.transform( ava_train.loc[:,'Bin Internet Service':'Bin Paperless Billing'] )

In [ ]:
np.concatenate([X_imp_conti, X_imp_bin], axis=1).shape

(4226, 18)

In [137]:
# Method 1: only conti
X_imp = X_imp_conti

In [104]:
# Method 2: conti+bin
X_imp = np.concatenate([X_imp_conti, X_imp_bin], axis=1)

In [133]:
y = np.array( ava_train['Churn Category'] )

Borderline SMOTE exclusive block

In [76]:
X_imp = np.array( ava_train[['Satisfaction Score', 'Contract_Month-to-Month', 'Tenure in Months',
       'Number of Referrals', 'Total Long Distance Charges',
       'Number of Dependents', 'Monthly Charge', 'Age', 'Bin Internet Service',
       'Internet Type_None', 'HasDependents', 'Internet Type_Fiber Optic',
       'Offer_Offer E', 'Bin Unlimited Data', 'City_San Diego',
       'Payment Method_Credit Card', 'Bin Online Security',
       'Bin Paperless Billing']] )

y = np.array( ava_train['Churn Category'] )

Check Dimension

In [138]:
X_imp.shape, y.shape

((4226, 9), (4226,))

In [135]:
def f1_score(precision, recall):
    return 2*(precision*recall)/(precision+recall)

In [136]:
def calc_precision_recall(y_true, y_pred):
    # Instantiate counters
    TP = 0
    FP = 0
    FN = 0

    # Determine whether each prediction is TP, FP, TN, or FN
    for i in range(len(y_true)): 
        if y_true[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_true[i]!=y_pred[i]:
           FP += 1
        if y_pred[i]==0 and y_true[i]!=y_pred[i]:
           FN += 1
    
    # Calculate true positive rate and false positive rate
    # Use try-except statements to avoid problem of dividing by 0
    # print(f'TP={TP} FP={FP} FN={FN}')
    try:
        precision = TP / (TP + FP)
    except:
        precision = 1
    
    try:
        recall = TP / (TP + FN)
    except:
        recall = 1

    return precision, recall

## SMOTE

In [ ]:
for c in [0.1,0.01,1,10]:
    for d in [1,3]:
        params = f'-t 1 -s 0 -c {c} -d {d}'
        m = svm_train(y[:14950], X_imp[:14950], params)
        p_label, p_acc, p_val = svm_predict(y[14950:], X_imp[14950:], m)
        precision, recall = calc_precision_recall(y[14950:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_SMOTE_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([params,p_acc[0],precision,recall,f1_score(precision,recall)])

## 10 features

In [ ]:
for c in [0.1,0.01,1,10]:
    for d in [1,3]:
        params = f'-t 1 -s 0 -c {c} -d {d}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_10_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([params,p_acc[0],precision,recall])

Accuracy = 74.5179% (541/726) (classification)
Accuracy = 71.7631% (521/726) (classification)
Accuracy = 74.5179% (541/726) (classification)
Accuracy = 79.7521% (579/726) (classification)
Accuracy = 75.7576% (550/726) (classification)
Accuracy = 48.0716% (349/726) (classification)
Accuracy = 75.6198% (549/726) (classification)
Accuracy = 51.9284% (377/726) (classification)


## Only Numerical

logger

- -t 0 -s 0 -c 4 -d 1
- 4m16s / Accuracy = 77.9614% (566/726) (classification)

In [ ]:
# 4m16s, 17% Validation Set --> Accuracy = 77.9614% (566/726) (classification)
m = svm_train(y[:3500], X_imp[:3500], '-t 0 -s 0 -c 4 -d 1')
p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)

Accuracy = 77.9614% (566/726) (classification)


In [ ]:
for c in [0.1,0.01,1,10]:
    for d in [1,3]:
        params = f'-t 1 -s 0 -c {c} -d {d}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_num_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([params,p_acc[0],precision,recall])

Accuracy = 80.7163% (586/726) (classification)
Accuracy = 54.1322% (393/726) (classification)
Accuracy = 79.3388% (576/726) (classification)
Accuracy = 27.2727% (198/726) (classification)
Accuracy = 80.7163% (586/726) (classification)
Accuracy = 54.1322% (393/726) (classification)
Accuracy = 81.6804% (593/726) (classification)
Accuracy = 62.3967% (453/726) (classification)
Accuracy = 76.9972% (559/726) (classification)
Accuracy = 51.7906% (376/726) (classification)


In [ ]:
for c in [0.1,0.01,1,10]:
    for d in [1,3]:
        params = f'-t 1 -s 0 -c {c} -d {d}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_num_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([params,p_acc[0],precision,recall])

# Numerical + Binary

In [ ]:
for c in [0.01,0.1,1,10]:
    for d in [1,3]:
        # if c==0.1 and d==1: continue
        params = f'-t 1 -s 0 -c {c} -d {d}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_numbin_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([params,p_acc[0],precision,recall])

Accuracy = 72.7273% (528/726) (classification)
Accuracy = 64.1873% (466/726) (classification)
Accuracy = 79.8898% (580/726) (classification)
Accuracy = 63.3609% (460/726) (classification)
Accuracy = 80.1653% (582/726) (classification)
Accuracy = 61.157% (444/726) (classification)
Accuracy = 80.0275% (581/726) (classification)
Accuracy = 63.3609% (460/726) (classification)


In [ ]:
for c in [0.01,0.1,1,10]:
    for g in [0.1,1,10,100]:
        params = f'-t 2 -s 0 -c {c} -g {g}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_numbin_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            print(params)
            writer.writerow([params,p_acc[0],precision,recall])

Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.01 -g 0.1
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.01 -g 1
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.01 -g 10
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.01 -g 100
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.1 -g 0.1
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.1 -g 1
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.1 -g 10
Accuracy = 74.2424% (539/726) (classification)
-t 2 -s 0 -c 0.1 -g 100
Accuracy = 75.4821% (548/726) (classification)
-t 2 -s 0 -c 1 -g 0.1
Accuracy = 74.6556% (542/726) (classification)
-t 2 -s 0 -c 1 -g 1
Accuracy = 74.6556% (542/726) (classification)
-t 2 -s 0 -c 1 -g 10
Accuracy = 74.6556% (542/726) (classification)
-t 2 -s 0 -c 1 -g 100
Accuracy = 75.3444% (547/726) (classification)
-t 2 -s 0 -c 10 -g 0.1
Accuracy = 74.9311% (544/726) (classification)
-t 2 -s 0 -c 10 -g 1
Accuracy = 74.5179% (54

In [ ]:
# for c in [0.01,0.1,1,10]:
for c in [10]:
    # for g in [0.1,1,10,100]:
        params = f'-t 0 -s 0 -c {c}'
        m = svm_train(y[:3500], X_imp[:3500], params)
        p_label, p_acc, p_val = svm_predict(y[3500:], X_imp[3500:], m)
        precision, recall = calc_precision_recall(y[3500:], p_label)
        with open('/content/drive/MyDrive/MachineLearning/Selected/SVM_numbin_log.csv', mode='a', newline='\n') as f:
            writer = csv.writer(f)
            print(params)
            writer.writerow([params,p_acc[0],precision,recall])

Accuracy = 81.405% (591/726) (classification)
-t 0 -s 0 -c 10


# Use best param for full model

In [107]:
ava_train.columns

Index(['Satisfaction Score', 'Contract', 'Tenure in Months',
       'Number of Referrals', 'Total Long Distance Charges',
       'Number of Dependents', 'Monthly Charge', 'Age', 'Bin Internet Service',
       'Internet Type_None', 'HasDependents', 'Internet Type_Fiber Optic',
       'Offer_Offer E', 'Bin Unlimited Data', 'City_San Diego',
       'Payment Method_Credit Card', 'Bin Online Security',
       'Bin Paperless Billing', 'Churn Category'],
      dtype='object')

In [39]:
y_test = np.array([0]*1409)

In [116]:
X_test = test[['Satisfaction Score', 'Contract', 'Tenure in Months',
       'Number of Referrals', 'Total Long Distance Charges',
       'Number of Dependents', 'Monthly Charge', 'Age', 'Bin Internet Service',
       'Internet Type_None', 'HasDependents', 'Internet Type_Fiber Optic',
       'Offer_Offer E', 'Bin Unlimited Data', 'City_San Diego',
       'Payment Method_Credit Card', 'Bin Online Security',
       'Bin Paperless Billing']]

In [141]:
X_test = test

In [143]:
X_test

,Customer ID,Satisfaction Score,Contract,Number of Referrals,Latitude,City_San Diego,Tenure in Months,Bin Online Security,Internet Type_None,Age,Churn Category
0,9938-EKRGF,3.0,1.0,1.0,33.141265,0.0,47.0,NaN,0.0,73.0,NaN
1,7379-POKDZ,4.0,24.0,0.0,34.231318,0.0,50.0,0.0,1.0,64.0,NaN
2,0654-HMSHN,5.0,12.0,9.0,32.802959,1.0,52.0,1.0,0.0,NaN,NaN
3,2045-BMBTJ,NaN,1.0,1.0,NaN,NaN,11.0,0.0,1.0,35.0,NaN
4,0701-TJSEF,NaN,1.0,1.0,33.581045,0.0,13.0,0.0,0.0,67.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1404,4587-VVTOX,4.0,12.0,0.0,37.871416,0.0,15.0,0.0,1.0,49.0,NaN
1405,7716-YTYHG,NaN,NaN,NaN,40.448632,0.0,NaN,NaN,NaN,78.0,NaN
1406,7649-PHJVR,1.0,1.0,1.0,NaN,NaN,21.0,0.0,0.0,27.0,NaN
1407,7855-DIWPO,NaN,12.0,1.0,33.688546,0.0,24.0,0.0,1.0,38.0,NaN


In [144]:
# only continuous
imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(np.array(X_test.loc[:,'Satisfaction Score':'Age']))
X_test_imp = imp.transform( np.array(X_test.loc[:,'Satisfaction Score':'Age']) )

In [145]:
X_test_imp.shape

(1409, 9)

In [146]:
y.shape, X_imp.shape

((4226,), (4226, 9))

In [147]:
params = '-t 1 -s 0 -c 0.01 -d 3'
m = svm_train(y, X_imp, params)
p_label, p_acc, p_val = svm_predict(y_test, X_test_imp, m)

Accuracy = 82.2569% (1159/1409) (classification)


In [148]:
df = pd.concat( (test['Customer ID'], pd.DataFrame(columns=['Churn Category'], data=p_label)) , axis=1 )
df['Churn Category'] = df['Churn Category'].astype(int)
df.to_csv('/content/drive/MyDrive/MachineLearning/svm_D4.csv', index=False)

In [ ]:
p_label

In [74]:
df['Churn Category'].describe()

count    1409.000000
mean        1.056068
std         1.703224
min         0.000000
25%         0.000000
50%         0.000000
75%         3.000000
max         5.000000
Name: Churn Category, dtype: float64

# Model Training

In [ ]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer

In [ ]:
iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=101)

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

print(y_test)
print(poly_pred)

Code Ref: https://www.baeldung.com/cs/svm-multiclass-classification

In [ ]:
ava_train.columns

Index(['Customer ID', 'Age', 'Number of Dependents', 'Latitude', 'Longitude',
       'Satisfaction Score', 'Number of Referrals', 'Tenure in Months',
       'Avg Monthly Long Distance Charges', 'Avg Monthly GB Download',
       ...
       'Bin Streaming Music', 'Bin Unlimited Data', 'Contract_Month-to-Month',
       'Contract_One Year', 'Contract_Two Year', 'Bin Paperless Billing',
       'Payment Method_Bank Withdrawal', 'Payment Method_Credit Card',
       'Payment Method_Mailed Check', 'Churn Category'],
      dtype='object', length=1153)

In [ ]:
ava_train = ava_train[ava_train.columns.drop(list(ava_train.filter(regex='City')))] \
.drop(['Latitude', 'Longitude', 'IsMale', 'Bin Device Protection Plan', 'Bin Streaming TV', 'Bin Streaming Music', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges', 'Total Revenue'], axis=1)

ava_train = ava_train[ava_train.columns.drop(list(ava_train.filter(regex='Offer')))]

In [ ]:
ava_train

In [ ]:
X = ava_train.drop(['Customer ID', 'Churn Category'], axis=1)
normalized_X = (X-X.mean())/X.std()

In [ ]:
X = normalized_X[['Age','Number of Dependents','Satisfaction Score','Number of Referrals','Tenure in Months','Avg Monthly GB Download']].to_numpy()
y = ava_train['Churn Category'].to_numpy()

In [ ]:
imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(X)

In [ ]:
X_imp = imp.transform(X)

In [ ]:
X_imp

array([[-5.81121164e-01, -4.84110353e-01, -1.90270799e+00,
        -6.27104418e-01,  1.05875299e-01, -2.12727876e-01],
       [ 4.28467892e-01, -4.84110353e-01, -1.90270799e+00,
        -6.27104418e-01, -1.27920447e+00,  4.26882529e-01],
       [ 1.22961660e-16, -5.84400450e-17,  3.41078117e-17,
         7.35554989e-01,  1.05875299e-01,  5.42821859e-17],
       ...,
       [ 7.84793441e-01,  1.62658458e+00,  6.22505369e-01,
        -6.27104418e-01, -4.64451663e-01,  4.26882529e-01],
       [ 1.22961660e-16,  2.68193205e+00,  3.41078117e-17,
        -6.27104418e-01, -6.68139864e-01,  5.42821859e-17],
       [ 1.43805695e+00, -4.84110353e-01,  6.22505369e-01,
         2.09821440e+00, -1.03477863e+00, -4.58731877e-01]])

In [ ]:
X_imp.shape

(4226, 6)

In [ ]:
np.count_nonzero(~np.isnan(X_imp))

25356

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_imp, y, train_size=0.80, test_size=0.20, random_state=101)

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma=0.0001, C=1000).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=2).fit(X_train, y_train)

In [ ]:
poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

In [ ]:
poly_pred

In [ ]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  81.56
F1 (Polynomial Kernel):  75.43


In [ ]:
rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  81.56
F1 (RBF Kernel):  75.47


# Full Model

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma=0.0001, C=1000).fit(X_imp, y)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_imp, y)

In [ ]:
testX = test.drop(['Customer ID', 'Churn Category'], axis=1)
normalized_testX = (testX - testX.mean()) / testX.std()

In [ ]:
testX = normalized_testX[['Age','Number of Dependents','Satisfaction Score','Number of Referrals','Tenure in Months','Avg Monthly GB Download']].to_numpy()
# testX = test.drop(['Customer ID', 'Churn Category'], axis=1).to_numpy()

imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(testX)

testX_imp = imp.transform(testX)

In [ ]:
testX_imp.shape

(1409, 6)

In [ ]:
poly_pred = poly.predict(testX_imp)
rbf_pred = rbf.predict(testX_imp)

In [ ]:
poly_pred.astype(int).mean()

0.1596877217885025

In [ ]:
rbf_pred.astype(int).mean()

0.16962384669978708

## PolySVM Prediction

In [ ]:
tmp = pd.DataFrame(columns=['Churn Category'], data=poly_pred)
tmp

,Churn Category
0,4.0
1,4.0
2,0.0
3,4.0
4,5.0
...,...
1404,4.0
1405,3.0
1406,1.0
1407,0.0


In [ ]:
test['Customer ID']

0       9938-EKRGF
1       7379-POKDZ
2       0654-HMSHN
3       2045-BMBTJ
4       0701-TJSEF
           ...    
1404    4587-VVTOX
1405    7716-YTYHG
1406    7649-PHJVR
1407    7855-DIWPO
1408    8197-BFWVU
Name: Customer ID, Length: 1409, dtype: object

In [ ]:
xxx = pd.concat( (test['Customer ID'], tmp) , axis=1 )
xxx['Churn Category'] = xxx['Churn Category'].astype(int)

xxx.to_csv('poly_svm.csv', index=False)

## RbfSVM Prediction

In [ ]:
tmp = pd.DataFrame(columns=['Churn Category'], data=rbf_pred)
tmp

,Churn Category
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1404,0.0
1405,0.0
1406,1.0
1407,0.0


In [ ]:
xxx = pd.concat( (test['Customer ID'], tmp) , axis=1 )
xxx['Churn Category'] = xxx['Churn Category'].astype(int)

xxx.to_csv('rbf_svm.csv', index=False)

# Other


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
def create_models(seed=2):
    '''
    Create a list of machine learning models.
            Parameters:
                    seed (integer): random seed of the models
            Returns:
                    models (list): list containing the models
    '''

    models = []
    models.append(('dummy_classifier', DummyClassifier(random_state=seed, strategy='most_frequent')))
    models.append(('k_nearest_neighbors', KNeighborsClassifier(n_neighbors=6)))
    models.append(('logistic_regression', LogisticRegression(random_state=seed)))
    models.append(('support_vector_machines', SVC(random_state=seed)))
    models.append(('random_forest', RandomForestClassifier(random_state=seed)))
    models.append(('gradient_boosting', GradientBoostingClassifier(random_state=seed)))
    
    return models

# create a list with all the algorithms we are going to assess
models = create_models()

In [ ]:
imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(X)
X_imp = imp.transform(X)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_imp, y, train_size=0.75, test_size=0.25, random_state=101)

In [ ]:
X_train.shape

(3169, 6)

In [ ]:
X_test.shape

(1057, 6)

In [ ]:
# test the accuracy of each model using default hyperparameters
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    # fit the model with the training data
    model.fit(X_imp, y).predict(X_test)
    # make predictions with the testing data
    predictions = model.predict(X_test)
    # print(predictions.mean())
    # calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    # append the model name and the accuracy to the lists
    results.append(accuracy)
    names.append(name)
    # print classifier accuracy
    print('Classifier: {}, Accuracy: {})'.format(name, accuracy))

0.0
Classifier: dummy_classifier, Accuracy: 0.7350993377483444)
0.24219489120151372
Classifier: k_nearest_neighbors, Accuracy: 0.8003784295175024)
0.15799432355723747
Classifier: logistic_regression, Accuracy: 0.8098391674550615)
0.14758751182592242
Classifier: support_vector_machines, Accuracy: 0.8098391674550615)
0.36140018921475875
Classifier: random_forest, Accuracy: 0.7928098391674551)
0.2696310312204352
Classifier: gradient_boosting, Accuracy: 0.7984862819299905)


In [ ]:
testX = test.drop(['Customer ID', 'Churn Category'], axis=1)
normalized_testX = (testX - testX.mean()) / testX.std()

testX = normalized_testX[['Age','Number of Dependents','Satisfaction Score','Number of Referrals','Tenure in Months','Avg Monthly GB Download']].to_numpy()
# testX = test.drop(['Customer ID', 'Churn Category'], axis=1).to_numpy()

imp = SimpleImputer( missing_values = np.nan, strategy='mean' )
imp = imp.fit(testX)

testX_imp = imp.transform(testX)

In [ ]:
# Logistic
pred = models[2][1].fit(X_imp, y).predict(testX_imp)

tmp = pd.DataFrame(columns=['Churn Category'], data=pred)
tmp

,Churn Category
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1404,0.0
1405,0.0
1406,1.0
1407,0.0


In [ ]:
tmp.describe()

,Churn Category
count,1409.000000
mean,0.171753
std,0.384755
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,3.000000


In [ ]:
test['Customer ID']

0       9938-EKRGF
1       7379-POKDZ
2       0654-HMSHN
3       2045-BMBTJ
4       0701-TJSEF
           ...    
1404    4587-VVTOX
1405    7716-YTYHG
1406    7649-PHJVR
1407    7855-DIWPO
1408    8197-BFWVU
Name: Customer ID, Length: 1409, dtype: object

In [ ]:
xxx = pd.concat( (test['Customer ID'], tmp) , axis=1 )
xxx['Churn Category'] = xxx['Churn Category'].astype(int)

xxx.to_csv('logistic.csv', index=False)